### Produce Driver Standings

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
display(dbutils.fs.ls(presentation_folder_path))

In [0]:
race_results_list = spark.read.parquet(f'{presentation_folder_path}/race_results')\
    .filter(f"file_date = '{v_file_date}'")\
    .select("race_year")\
    .distinct()\
    .collect()

In [0]:
race_year_list = []
for race_year in race_results_list:
    race_year_list.append(race_year.race_year)

In [0]:
from pyspark.sql.functions import col

race_results_df = spark.read.parquet(f'{presentation_folder_path}/race_results')\
    .filter(col('race_year').isin(race_year_list))

In [0]:
display(race_results_df)

In [0]:
from pyspark.sql.functions import sum,countDistinct,when,col,count, rank

In [0]:
driver_standing_df = race_results_df\
    .groupBy('race_year','driver_name','driver_nationality','team')\
    .agg(sum('points').alias('total_points'),\
    count(when(col('position')==1,True)).alias('wins')
    )

In [0]:
display(driver_standing_df.filter('race_year == 2020'))

In [0]:
from pyspark.sql.window import Window

In [0]:
driver_rank_spec = Window.partitionBy('race_year').orderBy(col('total_points').desc(), col('wins').desc())

In [0]:
final_df = driver_standing_df.withColumn('rank', rank().over(driver_rank_spec))

In [0]:
display(final_df.filter('race_year == 2020'))

In [0]:
# final_df.write.mode('overwrite').format('parquet').saveAsTable('f1_presentation.driver_standings')

In [0]:
overwrite_partition(final_df, 'f1_presentation', 'driver_standings', 'race_year')

In [0]:
%sql
SELECT COUNT(DISTINCT(race_year)) FROM f1_presentation.driver_standings